In [15]:
import dask.dataframe as dd
from dask.distributed import Client
import pandas as pd

In [16]:
client = Client(memory_limit='100GB',n_workers=1)
client

D:\ML_projects\mts\venv\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52786 instead
  warnings.warn(
2023-01-30 23:12:47,347 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.88 GiB


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:52786/status,
Dashboard: http://127.0.0.1:52786/status,Workers: 1
Total threads: 12,Total memory: 31.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52789,Workers: 1
Dashboard: http://127.0.0.1:52786/status,Total threads: 12
Started: Just now,Total memory: 31.88 GiB
Comm: tcp://127.0.0.1:52796,Total threads: 12
Dashboard: http://127.0.0.1:52797/status,Memory: 31.88 GiB
Nanny: tcp://127.0.0.1:52792,


In [32]:
df = dd.read_parquet('data/region_name.parquet')
df_2 = dd.read_parquet('data/city_name.parquet')

In [33]:
df['city_name'] = df_2['city_name']
df.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 2 columns):
 #   Column       Dtype   
---  ------       -----   
 0   region_name  category
 1   city_name    category
dtypes: category(2)
memory usage: 923.8 MB
